# Marginal likelihood timing tests

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import starry
import theano
import theano.tensor as tt
import itertools
import time

starry.config.lazy = True
starry.config.quiet = True

In [ ]:
map5 = starry.Map(ydeg=5)
map10 = starry.Map(ydeg=10)

In [ ]:
def get_func_and_grad(ydeg, npts, C, L, woodbury):

    if ydeg == 5:
        map = map5
    else:
        map = map10

    flux = np.ones(npts)
    theta = np.linspace(0, 360, npts)

    if C == "scalar":
        C = np.array(1)
    elif C == "vector":
        C = np.ones(npts)
    else:
        C = np.eye(npts)

    if L == "scalar":
        L = np.array(1.0)
    elif L == "vector":
        L = np.ones(map.Ny)
    else:
        L = np.eye(map.Ny)

    map.set_data(flux, C=C)
    map.set_prior(L=L)

    def _lnlike(inc):
        map.inc = inc
        return map.lnlike(theta=theta, woodbury=woodbury)

    # Compile the theano functions
    inc = tt.dscalar()
    func = theano.function([inc], _lnlike(inc), profile=True)
    grad = theano.function([inc], theano.grad(_lnlike(inc), inc), profile=True)
    return func, grad

In [ ]:
params = list(
    itertools.product(
        [5, 10],
        [1, 3, 10, 30, 100, 300, 1000],
        ["scalar", "vector", "matrix"],
        ["scalar", "vector", "matrix"],
        [False, True],
    )
)

In [ ]:
ydeg = np.array([p[0] for p in params])
npts = np.array([p[1] for p in params])
C = np.array([p[2] for p in params])
L = np.array([p[3] for p in params])
woodbury = np.array([p[4] for p in params])

In [ ]:
def time_func_and_grad(*args, nruns=10):
    func, grad = get_func_and_grad(*args)
    tfunc = np.zeros(nruns)
    tgrad = np.zeros(nruns)
    for k in range(nruns):
        tstart = time.time()
        func(30.0)
        tfunc[k] = time.time() - tstart
        tstart = time.time()
        grad(30.0)
        tgrad[k] = time.time() - tstart
    tfunc = np.median(tfunc)
    tgrad = np.median(tgrad)
    return tfunc, tgrad


tfunc = np.zeros(len(params))
tgrad = np.zeros(len(params))
for i, args in enumerate(params):
    tfunc[i], tgrad[i] = time_func_and_grad(*args)

In [ ]:
for l in [5, 10]:

    plt.figure(figsize=(12, 7))

    i = woodbury & (ydeg == l)
    plt.plot(npts[i], tfunc[i], "C0o")
    plt.plot(npts[i], tfunc[i], "C0-", lw=2, alpha=0.5)
    plt.plot(npts[i], tgrad[i], "C0^")
    plt.plot(npts[i], tgrad[i], "C0-", lw=2, alpha=0.5)

    i = np.invert(woodbury) & (ydeg == l)
    plt.plot(npts[i], tfunc[i], "C1o")
    plt.plot(npts[i], tfunc[i], "C1-", lw=2, alpha=0.5)
    plt.plot(npts[i], tgrad[i], "C1^")
    plt.plot(npts[i], tgrad[i], "C1-", lw=2, alpha=0.5)

    plt.xscale("log")
    plt.yscale("log")
    plt.ylim(1e-4, 1e-1)

    plt.plot([0], [0], "ko", label="func")
    plt.plot([0], [0], "k^", label="grad")
    plt.plot([0], [0], "C0-", label="woodbury")
    plt.plot([0], [0], "C1-", label="standard")
    plt.legend(fontsize=10)

    plt.xlabel("number of light curve points")
    plt.ylabel("time per function call [s]")

    plt.title("Degree {}".format(l), fontsize=24)